In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from random import randint

from busca.classes import ALTURAS, COLUNAS
from busca.classes.gerente import GerenteRemocao
from busca.classes.patio import Container, Pilha, Patio

NCONTAINERS = 20000